### Let's try finding Venus' ephemerides in the ICRF coordinate system.

Small test here:

In [1]:
from skyfield.api import load
from skyfield import units
from astropy.time import Time

# Create a timescale and initialise beginning and end time.
ts = load.timescale()
t0 = Time('2011-08-18T10:00:00',scale ='utc')
t0= ts.from_astropy(t0)
print(t0.utc_jpl())
t1= ts.utc(2013)

# Load the JPL ephemeris DE421 (covers 1900-2050).
planets = load('de421.bsp')
venus = planets['venus']
sun = planets['sun']

# print(planets)
# print(venus)


A.D. 2011-Aug-18 10:00:00.0000 UTC


In [2]:
# The position of Venus, viewed from BCRS
venus_pos = venus.at(t0)
ra, dec, distance = venus_pos.radec() 

print(ra)
print(dec)
print(distance)
print(venus_pos)


09h 59m 15.86s
+15deg 40' 26.7"
0.72137 au
<Barycentric BCRS position and velocity at date t center=0 target=299>


In [3]:
# Let's convert this in degrees

import astropy.units as u

dec = dec.to(u.deg)
ra = ra.to(u.deg)

print("Position of Venus on the 18th august 2011 at 10am : ",dec, ra)

Position of Venus on the 18th august 2011 at 10am :  15.674088161227528 deg 149.81610096113602 deg


## Let's try the real thing now
Ok, now that we know how to compute ephemerides of Venus in degrees in the ICRF, let's try to send a job to ODA to 
find an image of venus with IBIS. What's INTEGRAL center of coordinate system?-> we'll work with astropy to avoid
confusion between date format and coordinate systems.

We know that IBIS has quite a large FoV so that serendipitous observations of Venus is possible. 
The goal now is to try find a moment when IBIS was observing a portion of the sky where Venus was.

Fully-coded IBIS FoV: 8.3°x8°. Down to zero: 29.1°x29.4°. What's the portion of the sky that this covers ? 
Whole sky is about 41'000 $deg^2$ so the FoV covers between 66.4 and 855.5 square degrees. 
The last number is quite elevated. Chances to find Venus are actually high I think.

What's Venus's apparent size in the sky on average ? Do we want to compute a probability of finding it ?
As in the 2001 paper, what could be interesting would be to find when Venus was close to its maximum elongation so that we have less 
pollution from the Sun.


## Finding Venus' max elongation.

In [16]:
from skyfield.api import load
from skyfield.framelib import ecliptic_frame
from skyfield.searchlib import find_maxima
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord

ts = load.timescale()
t0 = ts.from_astropy(Time('2022-04-18T00:00:00',scale ='utc'))
t1 = ts.from_astropy(Time('2022-04-25T00:00:00',scale ='utc'))

eph = load('de421.bsp')
sun, earth, venus = eph['sun'], eph['earth'], eph['venus']

def elongation_at(t):
    e = earth.at(t)
    s = e.observe(sun).apparent()
    v = e.observe(venus).apparent()
    return s.separation_from(v).degrees

elongation_at.step_days = 1

print(elongation_at(t0), elongation_at(t1))

44.46437892273577 43.479544293232905


Let's compute the positions for all these 17 maximum elongations.

In [6]:
venus_pos=venus.at(times)
ra, dec, distance = venus_pos.radec() 
dec = dec.to(u.deg)
ra = ra.to(u.deg)
is_east=-1
for dec_obj, ra_obj,t,elongation_degrees in zip(dec, ra,times,elongations):
    is_east=(is_east+1)%2
    direction = 'east' if is_east else 'west'
    print('Position of Venus on the {} for a {:4.1f}° {} elongation: RA: {:4.5f}, Dec: {:4.5f}'.format(t.utc_strftime(), elongation_degrees, direction, ra_obj,dec_obj))

Now we try to find scw for these position and dates. I'll try with the web interface first. What should I put in source name ?
We'll first try with 2012-03-27 07:44:19 UTC and position RA: 145.37028 deg, Dec: 17.09323 deg.

In [90]:
import astroquery.heasarc
from astroquery.heasarc import Heasarc, Conf
import json
import os
import shutil
import random
from astropy.io import fits
from scipy import stats

Heasarc = astroquery.heasarc.Heasarc()
Conf.server.set('https://www.isdc.unige.ch/browse/w3query.pl')
table = Heasarc.query_mission_list()
table.pprint(max_width=120)

def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'), #--------> this function is deprecated. See below.
            radius = f"{radius} deg",
            mission = 'integral_rev3_scw',
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )

    #R.sort('SCW_ID')

    return R #R['SCW_ID'], R['SCW_VER']

assert astroquery.__version__ >= '0.4.2.dev6611'

# it means it's our fork
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

radius = 10.
ra_test = ra[7]
dec_test = dec[7]
t_test = times[7]
t_in = times[7]
coord = SkyCoord(ra_test, dec_test, frame='icrs',unit='deg')
print(ra_test, dec_test,t_test.utc_strftime(),t_test.utc_iso(),times[7], coord)

   Mission            Table                         Table Description               
------------- ---------------------- -----------------------------------------------
CTASST1M-REV1     cta_sst1m_rev1_run                                             Run
    FACT-REV1          fact_rev1_run                                             Run
INTEGRAL-REV3     integral_rev3_prop                                       Proposals
INTEGRAL-REV3 integral_rev3_prop_obs Proposal Information and Observation Parameters
INTEGRAL-REV3      integral_rev3_scw                       SCW - Science Window Data
145.3702781997043 deg 17.09322558279773 deg 2012-03-27 07:44:19 UTC 2012-03-27T07:44:19Z <Time tt=2456013.823212227> <SkyCoord (ICRS): (ra, dec) in deg
    (145.3702782, 17.09322558)>


In [105]:
table = Heasarc.query_region(coord, mission = 'integral_rev3_scw', radius='15 degree', time = '2012-01-01T00:30:00 .. 2012-12-31T07:50:00')
print(table)
file = open('venus_scw.txt','w')
file.write(str(table))
file.close()

   SCW_ID    SCW_VER ...                   SEARCH_OFFSET_                 
------------ ------- ... -------------------------------------------------
122500520021 001     ... 145.216 (145.37028360383465,17.093220361563155)\n
122500520010 001     ... 146.549 (145.37028360383465,17.093220361563155)\n
116900060010 001     ... 147.843 (145.37028360383465,17.093220361563155)\n
116900050041 001     ... 147.856 (145.37028360383465,17.093220361563155)\n
116900050030 001     ... 147.868 (145.37028360383465,17.093220361563155)\n
116900050020 001     ... 147.868 (145.37028360383465,17.093220361563155)\n
116900050010 001     ... 147.889 (145.37028360383465,17.093220361563155)\n
116900040021 001     ... 151.953 (145.37028360383465,17.093220361563155)\n
123700810010 001     ... 153.222 (145.37028360383465,17.093220361563155)\n
123700810021 001     ... 153.535 (145.37028360383465,17.093220361563155)\n
         ...     ... ...                                               ...
122500530010 001     ... 

Ok that's not the best idea... Better: compute venus proper motion

Let's try to find a suitable event by checking solar activity events so that we can first constrain the times we want. Ok so first we check space weather. Find a timeline, compute venus' (ra,dec), plot it, check where there is longest elongation, try to find a scw overlapping.

20.04.2022 seems to have a been a high solar activity period: https://www.spaceweatherlive.com/en/archive/2022/05/10/rsga.html